In [0]:
#!python -m spacy download pt_core_news_sm

In [11]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt")
artigo_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/My Drive/curso_word2vec/teste.csv")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:

nlp = spacy.load("pt_core_news_sm")

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens_exemplo = tokenizador(texto)
print(tokens_exemplo)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [0]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    
    vetor_resultante = np.zeros((1,300))
    
    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
    
    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens_exemplo, w2v_modelo_cbow)

In [10]:
vetor_texto.shape

(1, 300)

In [12]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape) 

(90000, 300)
(20513, 300)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

LR_cbow = LogisticRegression(max_iter = 800)
LR_cbow.fit(matriz_vetores_treino, artigo_treino.category)
categorias_previsao = LR_cbow.predict(matriz_vetores_teste)
resultado = classification_report(artigo_teste.category, categorias_previsao)
print(resultado)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.81      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [15]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape) 

(90000, 300)
(20513, 300)


In [16]:
LR_sg = LogisticRegression(max_iter = 800)
LR_sg.fit(matriz_vetores_treino_sg, artigo_treino.category)
categorias_previsao_sg = LR_sg.predict(matriz_vetores_teste_sg)
resultado_sg = classification_report(artigo_teste.category, categorias_previsao_sg)
print(resultado_sg)

              precision    recall  f1-score   support

     colunas       0.81      0.72      0.76      6103
   cotidiano       0.64      0.81      0.71      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.80      0.81     20513



In [0]:
import pickle

with open("/content/drive/My Drive/curso_word2vec/rl_cbow.pkl", "wb") as f:
    pickle.dump(LR_cbow, f)

In [0]:
with open("/content/drive/My Drive/curso_word2vec/rl_sg.pkl", "wb") as f:
    pickle.dump(LR_sg, f)